In [153]:
# 아이템기반 협업 필터링 구현

# 컨텐츠 기반 필터링
# 컨텐츠끼리 유사도 기반 추천 필터링
# 장르/키워드 정보를 벡터화 하여 유사도를 측정

# 유사도 측정 : 코사인 유사도
# 두 벡터의 각 직각=0, 180도 = -1, 0도=1


In [154]:
# 1. import 설정
import pandas as pd
import numpy as np
import pymysql

In [155]:
# 2-1. db 연동하기
# mysql에 data 가져오기

import pymysql
import pandas as pd

conn = pymysql.connect(host='localhost', user='kdigital', password='mysql',
                     db='firstkino', charset='utf8') 
# cursor = conn.cursor()

# sql 가져오기
with conn.cursor() as cursor:
    cursor.execute("SELECT genreCd, genreNm FROM Genre")
    genres = cursor.fetchall()
    cursor.execute("SELECT movieCd, genreCd FROM MovieGenre")
    genre_match = cursor.fetchall()
    cursor.execute("SELECT * FROM Movie")
    movie = cursor.fetchall()

conn.commit()

conn.close()


In [156]:
# genre, genre_match, movie 컬럼 dataframe으로 변환
genre_list = list(genres)
genre_df = pd.DataFrame(genre_list, columns=['genreCd', 'genreNm'])
genre_df

match_list = list(genre_match)
match_df = pd.DataFrame(match_list, columns=['movieCd', 'genreCd'])
match_df

movie_list = list((movie))
movie_df = pd.DataFrame(movie_list, columns=['movieCd','movieNm','movieNmEn','ShortContent','prdtYear','showTm','openDt','naverScore','imdbScore','watchGradeCd'])
# movie_df = movie_df['naverScore'].dropna()
movie_df

,movieCd,movieNm,movieNmEn,ShortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None
1,19188001,개의 일생,A Dog's Life,None,1918,None,0,None,None,None
2,19190002,여름밤의 기묘한 이야기,Unheimliche Geschichten,None,1919,None,0,None,None,None
3,19190003,재판장,The President,None,1919,None,0,None,None,None
4,19190007,의리적 구토,Fight For Justice,None,1919,None,19191027,None,None,None
...,...,...,...,...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,From end to end,None,2021,None,0,None,None,None
46914,20219978,볼레로 만들기,Structure: Making Bolero,None,2020,None,0,None,None,None
46915,20219984,어디에도 없는 시간,Where would we go,None,2021,None,0,None,None,None
46916,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None


In [157]:
# 데이터 병합하기
movie_merge = pd.merge(match_df, movie_df, on="movieCd")
movie_merge = pd.merge(movie_merge, genre_df, on="genreCd") 
movie_merge 

,movieCd,genreCd,movieNm,movieNmEn,ShortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,genreNm
0,19318004,1,엠,M,None,1931,None,0,None,None,None,스릴러
1,19328006,1,위험한 게임,The Most Dangerous Game,None,1932,None,0,None,None,None,스릴러
2,19338003,1,킹콩,King Kong,None,1933,None,0,4.5,None,None,스릴러
3,19348007,1,나는 비밀을 알고 있다,The Man Who Knew Too Much,None,1957,None,0,None,None,None,스릴러
4,19358005,1,39 계단,The 39 Steps,None,1935,None,0,None,None,None,스릴러
...,...,...,...,...,...,...,...,...,...,...,...,...
73025,20219775,18,관공서,The Office,None,1966,None,0,None,None,None,다큐멘터리
73026,20219788,18,엑스레이,X-Ray,None,1974,None,0,None,None,None,다큐멘터리
73027,20219805,18,이력서,Curriculum Vitae,None,1975,None,0,None,None,None,다큐멘터리
73028,20219820,18,한창나이 선녀님,Burning Flower,None,2021,None,20211020,None,None,None,다큐멘터리


In [158]:
movie_merge2 = movie_merge[['movieCd', 'movieNm', 'genreNm','naverScore']].dropna()
len(movie_merge2)

3813

In [164]:
movie_merge2 = movie_merge2.groupby('movieCd').agg({'movieNm':'first',
                                                   'genreNm': ','.join,
                                                   'naverScore':'first'}).reset_index()
len(movie_merge2)

2106

In [165]:
# 3. 문자열 genreNm 칼럼을 Count 기반으로 피처 벡터화 변환
# CountVectorizer = 단어 벡터화 카운트 정렬
# TfidfTransformer = TF-IDF로 단어 벡터화

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_score = count_vect.fit_transform(movie_merge2['genreNm'])

tfidf_transformer = TfidfTransformer()

genre_score = tfidf_transformer.fit_transform(genre_score)

print(genre_score.shape)



(2106, 138)


In [166]:
# 4. 리스트 객체 문자열 변경, count 피처 벡터화하기

from sklearn.metrics.pairwise import cosine_similarity

genre_score_sim = cosine_similarity(genre_score, genre_score)
genre_score_sim[0]


array([1.        , 0.16571743, 0.78406083, ..., 0.        , 0.16571743,
       0.        ])

In [168]:
# 5. 사용자가 평가한 특정영화 정보, 유사도 뽑아내기

def find_genre_score(df, sim_metrics, title_name, top_n=100):

    # 입력한 영화 인덱스
    movieNm = df[df['movieNm'] == title_name]
    movieNm_index = movieNm.index.values

    # 입력한 영화 유사도 데이터 프레임을 추가하기
    df["similarity"] = sim_metrics[movieNm_index, :].reshape(-1,1)

    # 유사도 내림차순 정렬(상위 인덱스 100개 추출)
    temp = df.sort_values(by="similarity", ascending=False)
    final_index = temp.index.values[ : top_n]

    return df.iloc[final_index]

In [171]:
# 6. 특정영화 유사한 장르 영화를 평점 순위별로 추천하기

# 6-1. 어벤져스 높은 유사도를 가지는 영화 순위별 정렬하기
similar_movie = find_genre_score(movie_merge2, genre_score_sim, '어벤져스', 100)
similar_movie = similar_movie[['movieCd', 'movieNm', 'similarity', 'naverScore', 'genreNm']]
# '어벤져스'와 장르별 높은 유사도를 가지는 영화
# 장르별 유사도를 가지는 영화 중 네이버 평점 순위별 정렬하기

movie_genre_score = similar_movie[['movieNm', 'similarity', 'naverScore', 'genreNm']].sort_values(["similarity", "naverScore"], ascending=(False, False))[:10]
movie_genre_score

,movieNm,similarity,naverScore,genreNm
2094,잭 스나이더의 저스티스 리그,1.0,4.7,"액션,SF"
1875,어벤져스: 엔드게임,1.0,4.7,"액션,SF"
74,로보캅,1.0,4.6,"액션,SF"
1184,로보캅,1.0,4.6,"액션,SF"
1607,가디언즈 오브 갤럭시 VOL. 2,1.0,4.5,"액션,SF"
1555,로건,1.0,4.5,"액션,SF"
1352,엣지 오브 투모로우,1.0,4.5,"액션,SF"
1085,토탈 리콜,1.0,4.4,"액션,SF"
1074,어벤져스,1.0,4.4,"액션,SF"
1460,앤트맨,1.0,4.3,"액션,SF"
